In [1]:
import ipywidgets as widgets
from IPython.display import display
from stravalib.client import Client
from datetime import datetime
from stravalib.exc import AccessUnauthorized
import matplotlib.pyplot as plt
import folium
import branca
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import logging
# Set logging level to suppress informational messages from stravalib
logging.getLogger('stravalib').setLevel(logging.ERROR)
import sys
import io
import re


# Staps Maps

Stats Maps allows you to visualize your Strava activities on a map, with the route colored by different metrics such as pace, altitude, cadence and more.

In [6]:
# Step 1: Direct the athlete to Strava's authorization page
CLIENT_ID = "99300"
CLIENT_SECRET = "3807d10d20fc27ccaf92f90bd7f51ec4c7d077cf"
REDIRECT_URI = "http://localhost/callback"  # This is a dummy URI
SCOPES = "activity:read"

oauth_url = f"https://www.strava.com/oauth/authorize?client_id={CLIENT_ID}&response_type=code&redirect_uri={REDIRECT_URI}&scope={SCOPES}"

# Create the HTML link
link_html = f'<a href="{oauth_url}" target="_blank">Click on the link to authorize on Strava</a>'

# Display the link
display(HTML(link_html))


In [7]:
# Step 2: Provide a text widget for the athlete to paste the full URL from Strava
code_input = widgets.Text(
    value='',
    placeholder='Enter the full callback URL from Strava',
    description='URL:',
)
display(code_input)

# Define a function to observe changes to code_input's value
def on_code_input_change(change):
    # Extract the code from the provided URL if a full URL is detected
    url = change['new']
    match = re.search(r'code=([\w\d]+)', url)
    if match:
        code = match.group(1)
        code_input.value = code  # Update the widget's value immediately

# Attach the observer to the code_input widget
code_input.observe(on_code_input_change, names='value')

# Date range input
start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)
end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False
)
display(start_date_picker, end_date_picker)


Text(value='', description='URL:', placeholder='Enter the full callback URL from Strava')

DatePicker(value=None, description='Start Date', step=1)

DatePicker(value=None, description='End Date', step=1)

In [8]:
class Stream:
    def __init__(self, data):
        self.data = data

def refresh_access_token(refresh_token):
    code = code_input.value
    client = Client()
    refresh_response = client.refresh_access_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, refresh_token=refresh_token)
    new_access_token = refresh_response['access_token']
    new_refresh_token = refresh_response['refresh_token']
    return new_access_token, new_refresh_token

def fetch_streams(client, activity_id):
    desired_streams = [
        'time', 'distance', 'latlng', 'altitude', 'velocity_smooth', 
        'heartrate', 'cadence', 'watts', 'temp', 'moving', 'grade_smooth'
    ]
    streams = client.get_activity_streams(activity_id, types=desired_streams, resolution='high')
    if 'velocity_smooth' in streams:
        pace_min_per_km = [(1000 / speed) / 60 if speed else 0 for speed in streams['velocity_smooth'].data]
        streams['pace_min_per_km'] = Stream(pace_min_per_km)
    return streams

In [9]:
activity_dropdown = widgets.Dropdown(
    options=[('Select an activity', None)],
    description='Activity:',
    disabled=False,
    value=None
)

metric_dropdown = widgets.Dropdown(
    options=[],
    description='Metric:',
    disabled=True,
)

# List to store fetched activities
fetched_activities = []

# Create an output widget to capture and display function outputs
output = widgets.Output()

def fetch_activities_and_streams(button):
    global fetched_activities
    with output:
        code = code_input.value
        client = Client()
        try:
            access_token_response = client.exchange_code_for_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, code=code)
            client.access_token = access_token_response['access_token']
        except AccessUnauthorized:
            new_access_token, _ = refresh_access_token(refresh_token)
            client.access_token = new_access_token

        start_date = start_date_picker.value
        end_date = end_date_picker.value
        if start_date:
            start_date = datetime.combine(start_date, datetime.min.time())
        if end_date:
            end_date = datetime.combine(end_date, datetime.min.time())
        
        if start_date and end_date:
            activity_summaries = list(client.get_activities(before=end_date, after=start_date))
        else:
            activity_summaries = list(client.get_activities())

        fetched_activities.clear()
        for summary in activity_summaries:
            detailed_activity = client.get_activity(summary.id)
            streams = fetch_streams(client, detailed_activity.id)
            detailed_activity.streams = streams
            fetched_activities.append(detailed_activity)

        activity_dropdown.options = [('Select an activity', None)] + [(activity.name, activity) for activity in fetched_activities]
        
        # Print the titles of fetched activities
        print("Fetched Activities:")
        for activity in fetched_activities:
            print(f"  - {activity.name}")

# Display the fetch button and the output widget
fetch_button = widgets.Button(description="Fetch Activities")
fetch_button.on_click(fetch_activities_and_streams)
display(fetch_button, output)

Button(description='Fetch Activities', style=ButtonStyle())

Output()

In [21]:
def update_metric_dropdown(change):
    if change['new']:
        activity = change['new']
        available_metrics = list(activity.streams.keys())
        metric_dropdown.options = available_metrics
        metric_dropdown.disabled = False
    else:
        metric_dropdown.options = []
        metric_dropdown.disabled = True

activity_dropdown.observe(update_metric_dropdown, names='value')

# Create an output widget to capture and display the map
map_output = widgets.Output()

def plot_on_map(button):
    with map_output:
        map_output.clear_output(wait=True)
        activity = activity_dropdown.value
        metric = metric_dropdown.value

        if not activity or not metric:
            print("Please select both an activity and a metric.")
            return

        latlng = activity.streams['latlng'].data
        latitudes = [coord[0] for coord in latlng]
        longitudes = [coord[1] for coord in latlng]
        metric_values = activity.streams[metric].data
        
        # Assuming you have an 'elevation' stream for elevation data
        elevation = activity.streams['altitude'].data
        
        # Normalize elevation for line width
        normalized_elevation = (elevation - np.min(elevation)) / (np.max(elevation) - np.min(elevation))
        line_widths = np.interp(normalized_elevation, [0, 1], [2, 10])  # Change [2, 10] to adjust min and max widths
        
        non_zero_values = [val for val in metric_values if val > 0]
        tenth_percentile = np.percentile(non_zero_values, 10)
        ninetieth_percentile = np.percentile(non_zero_values, 90)
        colormap = branca.colormap.linear._colormaps['RdPu_09'].scale(tenth_percentile, ninetieth_percentile)
        
        m = folium.Map(location=[latitudes[0], longitudes[0]],  tiles='Stamen Terrain', zoom_start=13)
        colormap.caption = f'{metric.capitalize()}'
        m.add_child(colormap)
        
        for i in range(1, len(latitudes)):
            folium.PolyLine([(latitudes[i-1], longitudes[i-1]), (latitudes[i], longitudes[i])], 
                            color=colormap(metric_values[i]), 
                            weight=line_widths[i]).add_to(m)
        
        display(m)

plot_button = widgets.Button(description="Plot on Map") 
description_label = widgets.Label(value="Line width is proportional to elevation (thin at low elevation, thick at high elevation)")
plot_button.on_click(plot_on_map)

display(activity_dropdown, metric_dropdown, plot_button, description_label, map_output)

Dropdown(description='Activity:', index=1, options=(('Select an activity', None), ('Whatup', <Activity id=9907…

Dropdown(description='Metric:', index=6, options=('temp', 'watts', 'moving', 'latlng', 'velocity_smooth', 'gra…

Button(description='Plot on Map', style=ButtonStyle())

Label(value='Line width is proportional to elevation (thin at low elevation, thick at high elevation)')

Output()

## Instructions

1. **Strava Authorization**:
   - Click on the provided link to authorize this tool to access your Strava activities.
   - After authorizing, you'll be redirected to a callback URL. Copy the entire URL.
   - Paste the copied URL into the 'URL' input box in this tool.
   
   &nbsp;
2. **Fetch Activities**:
   - Choose a date range to filter the activities you want to fetch.
   - Click the 'Fetch Activities' button.
   
   &nbsp;
3. **Select Activity and Metric**:
   - Once the activities are fetched, select an activity from the dropdown menu.
   - Next, choose a metric that you want to use for coloring the route on the map. This metric will be used to show variations along your route. For example, if you choose 'cadence', areas of your route with higher cadence will be colored differently from areas with lower cadence.

   &nbsp;

4. **Visualize**:
   - After selecting both an activity and a metric, click the 'Plot on Map' button.
   - You'll see a map with your activity's route. The route's color varies based on the metric you selected.

   &nbsp;

Feel free to explore different activities and metrics. If you have any questions or feedback, please reach out!
